In [136]:
# Importing necessary packages
import re
import time
import string
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from multiprocessing import Pool
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [137]:
# Config for scraping information from the website landing page
landing_page_config = {
    'url': {
        'class': '_mm360j',
        'attribute': 'href'
    }
}

# Config for scraping information from each listing page
internal_page_config = {
    'listing_name': {
        'class': '_fecoyn4'
    },
    'listing_type': {
        'class': '_tqmy57'
    },
    'star_rating': {
        'class': '_1ne5r4rt'
    },
    'price': {
        'class': '_1jo4hgw'
    },
    'review': {
        'class': '_162hp8xh'
    },
    'amenities': {
        'class': '_19xnuo97'
    },
    'num_reviews': {
        'class': '_1qf7wt4w'
    },
    'location_name': {
        'class': '_pbq7fmm'
    },
    'owner_info': {
        'class': 'tehcqxo.dir.dir-ltr'
    },
    'owner_details': {
        'class': '_88xxct'
    },
    'house_timings': {
        'class': 'c1lue5su.dir.dir-ltr'
    },
    'listing_highlights': {
        'class': '_1vjikx5'
    },
    'response_times': {
        'class': 'fhhmddr.dir.dir-ltr'
    }
}

# Link config
base_link = 'https://www.airbnb.com/s/'

city_names = ['Palo-Alto--CA--United-States/']
# , 'New-York--NY--United-States/', 'Washington--D.C.--USA/', 'College-Park--MD--United-States/', 'Palo-Alto--CA--United-States/', 'Dallas--TX--United-States/'

extension = '?items_offset='

In [ ]:
# Scraping the website
class AirbnbParser:
    def __init__(self):
        self.curr_city = None
        self.final_data = None
        pass

    def get_driver(self):
        """
        Returns a new instance of selenium webdriver
        """
        return webdriver.Chrome('/usr/local/bin/chromedriver')

    def get_next_link(self, curr_offset, city, offset=20):
        """
        Generates next link from the current parsed link
        """
        return ''.join([base_link, city, 'homes', extension]) + str(curr_offset + offset)

    def get_url_list_from_landing_page(self, start_index, city):
        """
        Scrapes the landing pages to get list of listing URLs
        """
        driver = self.get_driver()
        link = self.get_next_link(0, city, start_index)
        driver.get(link)
        for key, value in landing_page_config.items():
            class_name = value['class']
            try:
                WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, class_name)))
            except:
                pass
            url_list = driver.find_elements(By.CLASS_NAME, class_name)
            attribute = value['attribute']
            if key in self.final_data.keys():
                self.final_data[key] += [data_val.get_attribute(attribute) for data_val in url_list]
            else:
                self.final_data[key] = [data_val.get_attribute(attribute) for data_val in url_list]
            if not url_list:
                return False
        return True
    
    def get_data_from_listing_page(self, link):
        """
        Scrapes each internal page to get data of each listing
        """
        driver = self.get_driver()
        driver.get(link)
        result = {}
        for _, value in internal_page_config.items():
            try:
                WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, value['class'])))
            except:
                pass
        for key, value in internal_page_config.items():
            scraped_data = driver.find_elements(By.CLASS_NAME, value['class'])
            result[key] = [data_val.text for data_val in scraped_data]
        result['url'] = link
        result['city'] = self.curr_city
        return result
    
    def scrape_pages(self, curr_city):
        index = 0
        continue_scraping = self.get_url_list_from_landing_page(index, curr_city)
        index+=20
        while continue_scraping and index<281:
            continue_scraping = self.get_url_list_from_landing_page(index, curr_city)
            index+=20
        with Pool(5) as pool:
            result = pool.map(self.get_data_from_listing_page, self.final_data['url'])
        pool.close()
        pool.join()
        return result
            
    def start_scraping(self):
        final_data = []
        for city in city_names:
            self.final_data = {}
            self.curr_city = city
            final_data += self.scrape_pages(city)
        return final_data
            
scraper = AirbnbParser()
final_data = scraper.start_scraping()
listings_df = pd.DataFrame(final_data)
listings_df.to_csv('California.csv')

## Processing

In [160]:
# load ny data intp df
df_listings = pd.read_csv('NewYork.csv', sep = ',', error_bad_lines=False, index_col=0)

In [161]:
df_listings.head()

,listing_name,listing_type,star_rating,price,review,amenities,num_reviews,location_name,owner_info,owner_details,house_timings,listing_highlights,response_times,url,city
0,['Met Gallery stunner w/ maid & laundry'],['Entire rental unit hosted by Joshua\n2 guest...,['5.0'],"['', '$16\n / night']",['Cheryl\nNovember 2021\nJoshua and his family...,"['Kitchen', 'Fast wifi – 686 Mbps', '50"" HDTV ...","['(5 reviews)', 'New York, United States']","['New York, United States']",['Hosted by Joshua\nJoined in December 2014'],['Met Gallery stunner w/ maid & laundry\n5.0\n...,['House rules\nCheck-in: After 3:00 PM\nChecko...,['Entire home\nYou’ll have the apartment to yo...,['Response rate: 100%\nResponse time: within a...,https://www.airbnb.com/rooms/48369592?previous...,New-York--NY--United-States/
1,['DOUBLE SHARED BATHROOM'],['Room in boutique hotel hosted by Reservation...,['5.0'],"['', '$55\n / night']","[""Mike\nOctober 2021\nYou can get a mini fridg...","['Wifi', 'Free street parking', 'TV', 'Elevato...","['(3 reviews)', 'New York, United States']","['New York, United States']",['Hosted by Reservation Desk\nJoined in Decemb...,"[""DOUBLE SHARED BATHROOM\n5.0\n(3 reviews)\n·\...",['House rules\nCheck-in: After 3:00 PM\nChecko...,"[""Enhanced Clean\nThis Host committed to Airbn...",['Response rate: 93%\nResponse time: within an...,https://www.airbnb.com/rooms/25944708?previous...,New-York--NY--United-States/
2,['SINGLE SHARED BATHROOM'],['Room in boutique hotel hosted by Reservation...,['4.33'],"['', '$55\n / night']",['Cullen\nOctober 2020\nFun location. Room is ...,"['Wifi', 'Free street parking', 'TV', 'Elevato...","['(3 reviews)', 'New York, United States']","['New York, United States']",['Hosted by Reservation Desk\nJoined in Decemb...,"[""SINGLE SHARED BATHROOM\n4.33\n(3 reviews)\n·...",['House rules\nCheck-in: After 3:00 PM\nChecko...,"[""Enhanced Clean\nThis Host committed to Airbn...",['Response rate: 93%\nResponse time: within an...,https://www.airbnb.com/rooms/23061804?previous...,New-York--NY--United-States/
3,['Charming boutique apt 15 min from Manhattan'],['Entire rental unit hosted by Francis\n3 gues...,[''],"['', '$47\n / night']",['Jean-Francois\nSeptember 2021\nNice place at...,"['Kitchen', 'Wifi', 'HDTV with Hulu, Apple TV,...","['2 reviews', 'Queens, New York, United States']","['Queens, New York, United States']",['Hosted by Francis\nJoined in May 2017'],"[""Charming boutique apt 15 min from Manhattan\...",['House rules\nCheck-in: Flexible\nCheckout: 1...,['Entire home\nYou’ll have the apartment to yo...,['Response rate: 90%\nResponse time: within an...,https://www.airbnb.com/rooms/49649623?previous...,New-York--NY--United-States/
4,['Flatbush Hideaway - Quiet and close to subwa...,['Entire residential home hosted by Jason\n4 g...,['4.89'],"['', '$49\n / night']",['Anna\nSeptember 2021\nAlways my go to stay!'...,"['Kitchen', 'Wifi', 'Free parking on premises'...","['(118 reviews)', 'Brooklyn, New York, United ...","['Brooklyn, New York, United States']",['Hosted by Jason\nJoined in April 2016'],"[""Flatbush Hideaway - Quiet and close to subwa...",['House rules\nCheck-in: After 3:00 PM\nChecko...,['Entire home\nYou’ll have the house to yourse...,['Response rate: 80%\nResponse time: within an...,https://www.airbnb.com/rooms/31221559?previous...,New-York--NY--United-States/


In [162]:
# generate Id column
df_listings['listing_id'] = df_listings['url'].str.extract('\/rooms\/(\d+)')

In [163]:
# re-arrange columns 
column_names = list(df_listings.columns.values)
column_names.insert(0, column_names.pop())
df_listings = df_listings.reindex(columns=column_names)

In [164]:
df_listings.head()

,listing_id,listing_name,listing_type,star_rating,price,review,amenities,num_reviews,location_name,owner_info,owner_details,house_timings,listing_highlights,response_times,url,city
0,48369592,['Met Gallery stunner w/ maid & laundry'],['Entire rental unit hosted by Joshua\n2 guest...,['5.0'],"['', '$16\n / night']",['Cheryl\nNovember 2021\nJoshua and his family...,"['Kitchen', 'Fast wifi – 686 Mbps', '50"" HDTV ...","['(5 reviews)', 'New York, United States']","['New York, United States']",['Hosted by Joshua\nJoined in December 2014'],['Met Gallery stunner w/ maid & laundry\n5.0\n...,['House rules\nCheck-in: After 3:00 PM\nChecko...,['Entire home\nYou’ll have the apartment to yo...,['Response rate: 100%\nResponse time: within a...,https://www.airbnb.com/rooms/48369592?previous...,New-York--NY--United-States/
1,25944708,['DOUBLE SHARED BATHROOM'],['Room in boutique hotel hosted by Reservation...,['5.0'],"['', '$55\n / night']","[""Mike\nOctober 2021\nYou can get a mini fridg...","['Wifi', 'Free street parking', 'TV', 'Elevato...","['(3 reviews)', 'New York, United States']","['New York, United States']",['Hosted by Reservation Desk\nJoined in Decemb...,"[""DOUBLE SHARED BATHROOM\n5.0\n(3 reviews)\n·\...",['House rules\nCheck-in: After 3:00 PM\nChecko...,"[""Enhanced Clean\nThis Host committed to Airbn...",['Response rate: 93%\nResponse time: within an...,https://www.airbnb.com/rooms/25944708?previous...,New-York--NY--United-States/
2,23061804,['SINGLE SHARED BATHROOM'],['Room in boutique hotel hosted by Reservation...,['4.33'],"['', '$55\n / night']",['Cullen\nOctober 2020\nFun location. Room is ...,"['Wifi', 'Free street parking', 'TV', 'Elevato...","['(3 reviews)', 'New York, United States']","['New York, United States']",['Hosted by Reservation Desk\nJoined in Decemb...,"[""SINGLE SHARED BATHROOM\n4.33\n(3 reviews)\n·...",['House rules\nCheck-in: After 3:00 PM\nChecko...,"[""Enhanced Clean\nThis Host committed to Airbn...",['Response rate: 93%\nResponse time: within an...,https://www.airbnb.com/rooms/23061804?previous...,New-York--NY--United-States/
3,49649623,['Charming boutique apt 15 min from Manhattan'],['Entire rental unit hosted by Francis\n3 gues...,[''],"['', '$47\n / night']",['Jean-Francois\nSeptember 2021\nNice place at...,"['Kitchen', 'Wifi', 'HDTV with Hulu, Apple TV,...","['2 reviews', 'Queens, New York, United States']","['Queens, New York, United States']",['Hosted by Francis\nJoined in May 2017'],"[""Charming boutique apt 15 min from Manhattan\...",['House rules\nCheck-in: Flexible\nCheckout: 1...,['Entire home\nYou’ll have the apartment to yo...,['Response rate: 90%\nResponse time: within an...,https://www.airbnb.com/rooms/49649623?previous...,New-York--NY--United-States/
4,31221559,['Flatbush Hideaway - Quiet and close to subwa...,['Entire residential home hosted by Jason\n4 g...,['4.89'],"['', '$49\n / night']",['Anna\nSeptember 2021\nAlways my go to stay!'...,"['Kitchen', 'Wifi', 'Free parking on premises'...","['(118 reviews)', 'Brooklyn, New York, United ...","['Brooklyn, New York, United States']",['Hosted by Jason\nJoined in April 2016'],"[""Flatbush Hideaway - Quiet and close to subwa...",['House rules\nCheck-in: After 3:00 PM\nChecko...,['Entire home\nYou’ll have the house to yourse...,['Response rate: 80%\nResponse time: within an...,https://www.airbnb.com/rooms/31221559?previous...,New-York--NY--United-States/


In [165]:
# create df for amenities
df_amenities = df_listings[['listing_id', 'amenities']]
df_amenities.set_index('listing_id')
df_amenities.head()

,listing_id,amenities
0,48369592,"['Kitchen', 'Fast wifi – 686 Mbps', '50"" HDTV ..."
1,25944708,"['Wifi', 'Free street parking', 'TV', 'Elevato..."
2,23061804,"['Wifi', 'Free street parking', 'TV', 'Elevato..."
3,49649623,"['Kitchen', 'Wifi', 'HDTV with Hulu, Apple TV,..."
4,31221559,"['Kitchen', 'Wifi', 'Free parking on premises'..."


In [166]:
# process amenities further

# df_amenities.unstack()['amenities'].str.strip('[]').str.split(', ', expand=True).value_counts()
# res = df_amenities.set_index(['listing_id'])['amenities'].apply(pd.Series).stack()
# res = res.reset_index()
# res.columns = ['listing_id','level_1','amenities']
# res.drop(columns='level_1', inplace = True)
# res
# df_amenities['amenities'][0]
df_listings.drop('amenities', axis=1, inplace=True) 

In [167]:
# Clean price data
df_listings['price'] = df_listings['price'].str.extract('\$(\d*\.?\d*)').astype(float)

In [168]:
# check records having bad data for price

# def is_integer_num(n):
#     if isinstance(n, int):
#         return True
#     if isinstance(n, float):
#         return not n.is_integer()
#     return False
# df_listings[df_listings['price'].apply(is_integer_num)]

In [169]:
# clean star_rating 
df_listings['star_rating'] = df_listings['star_rating'].str.extract('(\d*\.\d*)').astype(float)
# fill na with mean

In [170]:
# number of reviews

df_listings['num_reviews'] = df_listings['num_reviews'].str.extract('(\d+)\s+review').astype(float)
# len(df_listings[df_listings['num_reviews'].isnull()])

In [171]:
def clean_unicode(listing_name):
    text_encode = listing_name.encode(encoding="ascii", errors="ignore")
    text_decode = text_encode.decode()
    return " ".join([word for word in text_decode.split()])

In [172]:
punct = set(string.punctuation) 

def clean_tags(listing_name):
    # removing mentions e.g @CityBoy12
    listing_name = re.sub("@\S+", "", listing_name)
    # remove $ signs
    listing_name = re.sub("\$", "", listing_name)
    # remove urls
    listing_name = re.sub("https?:\/\/.*[\r\n]*", "", listing_name)
    # remove hashtags
    listing_name = re.sub("#", "", listing_name)
    # remove punctations
    listing_name = "".join([ch for ch in listing_name if ch not in punct])
    return listing_name

In [173]:
# clean listing names
df_listings['listing_name'] = df_listings['listing_name'].apply(clean_unicode)
df_listings['listing_name'] = df_listings['listing_name'].apply(clean_tags)
df_listings

,listing_id,listing_name,listing_type,star_rating,price,review,num_reviews,location_name,owner_info,owner_details,house_timings,listing_highlights,response_times,url,city
0,48369592,Met Gallery stunner w maid laundry,['Entire rental unit hosted by Joshua\n2 guest...,5.00,16.0,['Cheryl\nNovember 2021\nJoshua and his family...,5.0,"['New York, United States']",['Hosted by Joshua\nJoined in December 2014'],['Met Gallery stunner w/ maid & laundry\n5.0\n...,['House rules\nCheck-in: After 3:00 PM\nChecko...,['Entire home\nYou’ll have the apartment to yo...,['Response rate: 100%\nResponse time: within a...,https://www.airbnb.com/rooms/48369592?previous...,New-York--NY--United-States/
1,25944708,DOUBLE SHARED BATHROOM,['Room in boutique hotel hosted by Reservation...,5.00,55.0,"[""Mike\nOctober 2021\nYou can get a mini fridg...",3.0,"['New York, United States']",['Hosted by Reservation Desk\nJoined in Decemb...,"[""DOUBLE SHARED BATHROOM\n5.0\n(3 reviews)\n·\...",['House rules\nCheck-in: After 3:00 PM\nChecko...,"[""Enhanced Clean\nThis Host committed to Airbn...",['Response rate: 93%\nResponse time: within an...,https://www.airbnb.com/rooms/25944708?previous...,New-York--NY--United-States/
2,23061804,SINGLE SHARED BATHROOM,['Room in boutique hotel hosted by Reservation...,4.33,55.0,['Cullen\nOctober 2020\nFun location. Room is ...,3.0,"['New York, United States']",['Hosted by Reservation Desk\nJoined in Decemb...,"[""SINGLE SHARED BATHROOM\n4.33\n(3 reviews)\n·...",['House rules\nCheck-in: After 3:00 PM\nChecko...,"[""Enhanced Clean\nThis Host committed to Airbn...",['Response rate: 93%\nResponse time: within an...,https://www.airbnb.com/rooms/23061804?previous...,New-York--NY--United-States/
3,49649623,Charming boutique apt 15 min from Manhattan,['Entire rental unit hosted by Francis\n3 gues...,NaN,47.0,['Jean-Francois\nSeptember 2021\nNice place at...,2.0,"['Queens, New York, United States']",['Hosted by Francis\nJoined in May 2017'],"[""Charming boutique apt 15 min from Manhattan\...",['House rules\nCheck-in: Flexible\nCheckout: 1...,['Entire home\nYou’ll have the apartment to yo...,['Response rate: 90%\nResponse time: within an...,https://www.airbnb.com/rooms/49649623?previous...,New-York--NY--United-States/
4,31221559,Flatbush Hideaway Quiet and close to subway,['Entire residential home hosted by Jason\n4 g...,4.89,49.0,['Anna\nSeptember 2021\nAlways my go to stay!'...,118.0,"['Brooklyn, New York, United States']",['Hosted by Jason\nJoined in April 2016'],"[""Flatbush Hideaway - Quiet and close to subwa...",['House rules\nCheck-in: After 3:00 PM\nChecko...,['Entire home\nYou’ll have the house to yourse...,['Response rate: 80%\nResponse time: within an...,https://www.airbnb.com/rooms/31221559?previous...,New-York--NY--United-States/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,6990,UES Beautiful Blue Room,['Private room in rental unit hosted by Cyn\n1...,4.88,62.0,"[""Constanţa\nJuly 2021\nI lived at Cynthia's p...",234.0,"['New York, United States']",['Hosted by Cyn\nJoined in May 2009'],"[""UES Beautiful Blue Room\n4.88\n(234 reviews)...",['House rules\nCheck-in: Flexible\nCheckout: 1...,"[""Enhanced Clean\nThis Host committed to Airbn...",['Response rate: 100%\nResponse time: within a...,https://www.airbnb.com/rooms/6990?previous_pag...,New-York--NY--United-States/
296,43315528,138 Bowery Studio,['Entire rental unit hosted by Jeniffer\n2 gue...,NaN,145.0,"['Tommy\nMarch 2021\nSuperb location, clean an...",1.0,"['New York, United States']",['Hosted by Jeniffer\nJoined in December 2015'],"[""138 Bowery Studio\n1 review\n·\nNew York, Un...",['House rules\nCheck-in: After 4:00 PM\nChecko...,['Entire home\nYou’ll have the apartment to yo...,['Response rate: 86%\nResponse time: within a ...,https://www.airbnb.com/rooms/43315528?previous...,New-York--NY--United-States/
297,52264778,Midtown West Hotel NYC Economy queen,['Room in hotel hosted by Joe\n2 guests · 1 be...,4.20,105.0,['Christine\nOctober 2021\nAwesom

In [174]:
df_listings['listing_type'][4]

"['Entire residential home hosted by Jason\\n4 guests · 1 bedroom · 3 beds · 1 bath']"

In [175]:
# clean listing type

df_listing_type = df_listings[['listing_id', 'listing_type']]

df_listing_type[['Type']] = df_listing_type['listing_type'].str.extract("\['(?P<type>[\w*\s*()]*)hosted by").copy()
df_listing_type[['Host']] = df_listing_type['listing_type'].str.extract("hosted by\s*([\w*\s*]*)").copy()
df_listing_type[['No_of_Guest']] = df_listing_type['listing_type'].str.extract("\\\\n(\d*)").copy()
df_listing_type[['No_of_Bedrooms']] = df_listing_type['listing_type'].str.extract(".\s*(\d*)\sbedroom[s]?").copy()
df_listing_type[['No_of_Beds']] = df_listing_type['listing_type'].str.extract(".\s*(\d*)\sbed[s]?\s").copy()
df_listing_type[['No_of_Baths']] = df_listing_type['listing_type'].str.extract(".\s*(\d*)\sbath[s]?").copy() # review for shared bath
df_listing_type[['Is_Studio']] = df_listing_type['listing_type'].str.extract(".\s*(Studio)").copy()
df_listing_type.drop('listing_type', axis=1, inplace=True)

C:\Users\Darren\anaconda3\lib\site-packages\pandas\core\frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
C:\Users\Darren\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [176]:
df_listing_type

,listing_id,Type,Host,No_of_Guest,No_of_Bedrooms,No_of_Beds,No_of_Baths,Is_Studio
0,48369592,Entire rental unit,Joshua,2,1,1,1,NaN
1,25944708,Room in boutique hotel,Reservation Desk,2,1,1,,NaN
2,23061804,Room in boutique hotel,Reservation Desk,1,1,1,,NaN
3,49649623,Entire rental unit,Francis,3,1,2,1,NaN
4,31221559,Entire residential home,Jason,4,1,3,1,NaN
...,...,...,...,...,...,...,...,...
295,6990,Private room in rental unit,Cyn,1,1,1,,NaN
296,43315528,Entire rental unit,Jeniffer,2,NaN,1,1,Studio
297,52264778,Room in hotel,Joe,2,1,1,,NaN
298,15817291,Private room in rental unit,Mark,2,1,1,,NaN


In [177]:
df_listings.drop('listing_type', axis=1, inplace=True) 

In [194]:
print(df_listings['response_times'][0])
print(df_listings['response_times'][2])
print(df_listings['response_times'][10])
print(df_listings['response_times'][200])
print(df_listings['response_times'][260])
print(df_listings['response_times'][299])

['Response rate: 100%\nResponse time: within an hour']
['Response rate: 93%\nResponse time: within an hour']
['Languages: English, Español\nResponse rate: 100%\nResponse time: within an hour']
['Languages: English, Español\nResponse rate: 100%\nResponse time: within an hour']
['Language: Español']
['Response rate: 97%\nResponse time: within an hour']


In [192]:
df_response_times = df_listings[['listing_id', 'response_times']]

df_response_times[['Languages']] = df_response_times['response_times'].str.extract("\['Languages:\s(.*)\\nResponse rate").copy() # review NaN ?
df_response_times[['Response Rate']] = df_response_times['response_times'].str.extract("nResponse rate: (\d*)").astype(float).copy()
df_response_times[['Response Time']] = df_response_times['response_times'].str.extract("\\nResponse time:\s([\w*\s]*)").copy() # review NaN ?